In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import random


TRAIN_DIR = "C:\\Users\\onuro\\Desktop\\Train_2"
TEST_DIR = "C:\\Users\\onuro\\Desktop\\Test_2"
CATEGORIES = ["Train_Cat", "Train_Dog"]
CATEGORIES_2 = ["Test_Cat", "Test_Dog"]
IMG_SIZE=375

for category in CATEGORIES:  
    path = os.path.join(TRAIN_DIR,category)  
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE) 
        
        break  # we just want one for now so break
    break  #...and one more!
    
for category_2 in CATEGORIES_2:  # do dogs and cats
    path = os.path.join(TEST_DIR,category_2)  # create path to dogs and cats
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array

        break  # we just want one for now so break
    break  #...and one more!



cat_dog = 'dogsvscats-{}-{}.model'.format(0.01, '2conv-basic') # just so we remember which saved model is which, sizes must match

In [2]:
def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'cat': return 1
    #                             [no cat, very doggo]
    elif word_label == 'dog': return -1

In [3]:
def create_training_data():
    training_data = []
    for category in CATEGORIES:  # do dogs and cats
        
        path = os.path.join(TRAIN_DIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (-1 or a 1). -1=dog 1=cat
        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            label = label_img(img)
            try:    
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                training_data.append([np.array(img_array), np.array(class_num)])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
    random.shuffle(training_data)
    np.save('train_data.npy', training_data)   
    return training_data
    
train_data=create_training_data()

print(len(train_data))

100%|██████████| 10120/10120 [00:59<00:00, 169.53it/s]


20239


In [4]:
def create_test_data():
    testing_data = []
    for category_2 in CATEGORIES_2:       
        path = os.path.join(TEST_DIR,category_2)
        class_num=CATEGORIES_2.index(category_2)
        
        for img in tqdm(os.listdir(path)):
            label = label_img(img)
            
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                testing_data.append([np.array(img_array), class_num])        
            except Exception as e:
                pass
            
    random.shuffle(testing_data) #karistirir
    np.save('test_data.npy', testing_data)    
    return testing_data   
test_data=create_test_data()

print(len(test_data))


100%|██████████| 3000/3000 [00:17<00:00, 173.71it/s]


6000


In [8]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
tf.reset_default_graph()

network = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

network = conv_2d(network, 16, 5,  activation='tanh')
network = max_pool_2d(network, 5)

network = conv_2d(network, 32, 5,   activation='tanh')
network= max_pool_2d(network, 5)

network = conv_2d(network, 64, 5,   activation='relu')
network = dropout(network, 0.8)

network = conv_2d(network, 128, 5,   activation='relu')
network = max_pool_2d(network, 5)

network = fully_connected(network, 1024, activation='relu')
network = dropout(network, 0.2)
network = fully_connected(network, 1, activation='softmax')

network = regression(network, optimizer='adam', learning_rate=1e-3,
                     loss='mean_square', name='target')

model = tflearn.DNN(network, tensorboard_dir='log')
print('okey')

okey


In [5]:
import numpy as np
X=np.array([i[0] for i in train_data]).reshape(-1,IMG_SIZE, IMG_SIZE,1)
Y=[i[1] for i in train_data]

Y= np.reshape(Y,(-1,1))


X_test = np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE, IMG_SIZE,1)
Y_test = [i[1] for i in test_data]
Y_test = np.reshape(Y_test, (-1, 1))


In [9]:
history=model.fit(X,Y, n_epoch=10, batch_size=100, validation_set=0.0001, snapshot_step=None,
         show_metric=True, run_id=cat_dog)

Training Step: 2029  | total loss: 0.50269 | time: 6843.588s
| Adam | epoch: 010 | loss: 0.50269 - binary_acc: 0.4973 -- iter: 20200/20236
Training Step: 2030  | total loss: 0.50342 | time: 6878.511s
| Adam | epoch: 010 | loss: 0.50342 - binary_acc: 0.4966 | val_loss: 0.00000 - val_acc: 1.0000 -- iter: 20236/20236
--


In [23]:
model = DNN(network, tensorboard_verbose=3)

In [5]:
!tensorboard --logdir='log/dogsvscats-0.01-2conv-basic.model'

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.0.2 at http://localhost:6006/ (Press CTRL+C to quit)
^C
